In [1]:
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd
from datetime import date

In [2]:
url = 'https://economia.uol.com.br/cotacoes/'

In [3]:
def Commodities():        
    response = requests.get(url)
    html = response.content
    soup = BeautifulSoup(html, 'html.parser')
    # Encontra div das 3 últimas tabelas do site, Inflação, Commodities e Produtos agrícolas
    divs = soup.findAll('div', attrs={'class':'col-lg-6 col-md-6 col-sm-12 col-xs-12 accordion'})

    res = []
    table = []

    # Para cada div, extrai o que tem na table do html (retorna as 3 tabelas)
    for div in divs:
        table.extend(div.find('table'))

     # Pega apenas a table[3], com os dados das Commodities e separa por tr do html 
    table_rows = table[3].find_all('tr')

    for tr in table_rows: # Para cada linha
        data = date.today() # Pega o dia atual
        data = data.strftime("%d/%m/%Y") # Coloca no formato dd/mm/yyyy

        td = tr.find_all('td') # Separa em listas cada conteúdo (td) da linha do html 
        row = [tr.text for tr in td]
        row = row + [data]
        if row:
            res.append(row)
    
    # Tenta ler a tabela já criada, se ainda não existe, cria uma nova
    try: 
        old = pd.read_excel('DadosCommodities.xlsx')
    except Exception:
        print("Arquivo 'DadosCommodities' ainda não criado, gerando novo...")
        vazio = []
        vazio = pd.DataFrame(vazio)
        vazio.to_excel('DadosCommodities.xlsx', index = False)
        old = pd.read_excel('DadosCommodities.xlsx')

    df = pd.DataFrame(res, columns=['Produto', 'Variacao', 'Venda', "Data"])
    df = df.dropna()
    df = df.append(old)
    df.drop_duplicates(subset=['Produto', 'Data'], keep='first', inplace=True)
    df.to_excel('DadosCommodities.xlsx', index = False)


In [4]:
Commodities()
pd.read_excel('DadosCommodities.xlsx')

,Produto,Variacao,Venda,Data
0,Petróleo,"+1,37%","US$ 72,510",21/12/2021
1,Ouro,"+0,25%","US$ 1798,400",21/12/2021
2,Prata,"+2,13%","US$ 22,700",21/12/2021
3,Platina,"+0,69%","US$ 932,800",21/12/2021
4,Paládio,"-2,62%","US$ 1757,500",21/12/2021
5,Petróleo,"-5,35%","US$ 69,600",20/12/2021
6,Ouro,"-0,57%","US$ 1793,800",20/12/2021
7,Prata,"-1,2%","US$ 22,275",20/12/2021
8,Platina,"-1,52%","US$ 920,500",20/12/2021
9,Paládio,"-2,62%","US$ 1751,500",20/12/2021


In [ ]:
# while True:
#     Commodities()
#     time.sleep(86400)